In [16]:
import pandas as pd
import onnx
import numpy as np
import tensorflow as tf
import pickle

#from tf2onnx.convert import convert
from skl2onnx.common.data_types import FloatTensorType
from skl2onnx import convert_sklearn
from sklearn.ensemble import RandomForestRegressor
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.metrics import mean_absolute_error, mean_squared_error
from sklearn.metrics import r2_score


dataset = ('Ethos_Dataset_Binary.csv')
df = pd.read_csv(dataset, sep=';') 

df.head()

,comment,isHate
0,You should know women's sports are a joke,1.0
1,You look like Sloth with deeper Down’s syndrome,1.0
2,You look like Russian and speak like Indian. B...,1.0
3,"Women deserve to be abused, I guess.",1.0
4,Women are made for making babies and cooking d...,1.0


In [2]:


X = df['comment']
Y = df['isHate']

X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2, random_state=42)

X.head()

0            You should know women's sports are a joke
1      You look like Sloth with deeper Down’s syndrome
2    You look like Russian and speak like Indian. B...
3                 Women deserve to be abused, I guess.
4    Women are made for making babies and cooking d...
Name: comment, dtype: object

In [3]:
vectorizer = TfidfVectorizer()
X = vectorizer.fit_transform(X)
X_train = vectorizer.fit_transform(X_train)
X_test = vectorizer.transform(X_test)

In [4]:
rf_model = RandomForestRegressor(n_estimators=100, random_state=42)
rf_model.fit(X_train, Y_train)

RandomForestRegressor(random_state=42)

In [5]:
comentario = 'thats not cool man' 
comentario = vectorizer.transform([comentario])

prediction = rf_model.predict(comentario)

print("nivel de hostilidade: ", prediction)

nivel de hostilidade:  [0.09666667]


In [6]:


# Fazer previsões
predictions = rf_model.predict(X_test)

# Calcular o coeficiente de determinação
r2 = r2_score(Y_test, predictions)

print("Coeficiente de determinação (R²): {:.4f}".format(r2))


Coeficiente de determinação (R²): 0.0778


In [7]:
# Calcular o MAE
mae = mean_absolute_error(Y_test, predictions)
print("MAE: {:.4f}".format(mae))

MAE: 0.3053


In [8]:
# Calcular o MSE
mse = mean_squared_error(Y_test, predictions)
print("MSE: {:.4f}".format(mse))

MSE: 0.1345


In [19]:
input_names = ['comment']
output_names = ['isHate']

initial_type = [('comment', FloatTensorType([None, X_train.shape[1]]))]

onnx_model = convert_sklearn(rf_model, initial_types=initial_type)

# Salvar os dados no arquivo pickle
with open('vectorizer.pkl', 'wb') as f:
    pickle.dump(vectorizer, f)

onnx.save_model(onnx_model, 'HateDetector_Trained_Model.onnx')